In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
tf.compat.v1.disable_eager_execution()

In [2]:
path = 'From_Vzhik/DJI_0290.JPG'
model_bin ='../git/ladd-and-weights/weights/yolo5/yolo5_fullDS_TF.pb'
network_in_size = 1984

In [3]:
def get_image_shape(path: str):
    img = cv2.imread(path) ## ??? is it the same way in C#? Quite inefficient
    return img.shape

In [4]:
def compute_resize_scale(image_shape):
    r = min(network_in_size / image_shape[0], network_in_size / image_shape[1])
    r = min(r, 1.0)
    return r

In [5]:
def read_image_tf(path: str, sess): #RGB
    h, w, _ = get_image_shape(path)
    scale = compute_resize_scale((h, w))
    new_unpad_size = [int(round(h * scale)), int(round(w * scale))] ## round!!!!

    file_reader = tf.io.read_file(filename=path, name="file_reader")
    decode_jpeg = tf.image.decode_jpeg(file_reader, channels=3, dct_method="INTEGER_ACCURATE",name="decode_jpeg")
    divider = tf.constant(255.0, dtype=tf.float32, name="divider")
    casted = tf.cast(decode_jpeg, dtype=tf.float32, name="cast")
    casted_normalized = tf.divide(casted, divider, name="normalized")

    dims_expander = tf.expand_dims(casted_normalized, 0, name="dims_expander")
    resize_jpeg = tf.compat.v1.image.resize_bilinear(dims_expander, new_unpad_size, half_pixel_centers=True, align_corners=False, name="resize")
    # Below yeilds the same results
    # resize_jpeg = tf.image.resize(dims_expander, new_unpad_size, method=tf.image.ResizeMethod.BILINEAR,name="resize")
    
    dh = (network_in_size - new_unpad_size[0])/2
    dw = (network_in_size - new_unpad_size[1])/2
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    pad = [[0, 0], [top,bottom], [left,right] ,[0, 0]]    
    image_pad = tf.pad(resize_jpeg, pad, mode = "CONSTANT", constant_values = 114.0/255.0, name="output")    
    
    return image_pad, w, h, top, left

In [6]:
def load_model_tf(model_bin: str, sess):
    with tf.io.gfile.GFile(model_bin, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
        return sess.graph

In [7]:
def infer_tf(image, model, sess):
    x = model.get_tensor_by_name('x:0')
    boxes = model.get_tensor_by_name('Identity:0')
    scores = model.get_tensor_by_name('Identity_1:0')
    classes = model.get_tensor_by_name('Identity_2:0')
    valid_detections = model.get_tensor_by_name('Identity_3:0')

    boxes, scores, classes, valid_detections = sess.run([boxes, scores, classes, valid_detections], feed_dict={
            x: image
        })
    return boxes, scores, classes, valid_detections

In [8]:
with tf.compat.v1.Session() as sess:
    image_tf, w, h, top, left = read_image_tf(path,sess)
    model = load_model_tf(model_bin, sess)
    boxes, scores, _, valid_detections = infer_tf(image_tf.eval(), model, sess)

2022-01-22 19:07:50.230980: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-22 19:07:50.231361: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-22 19:07:50.232822: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-01-22 19:07:51.037381: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2022-01-22 19:07:51.067263: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


In [9]:
def clip(x):
    return min(max((x),0.0),1.0)

In [10]:
def transform_predictions(boxes, scores, valid_detections, image_w, image_h,top,left):
    result = []
    for i in range(valid_detections[0]):
        x0=boxes[0][i][0]
        x1=boxes[0][i][2]
        y0=boxes[0][i][1]
        y1=boxes[0][i][3]
        # Adjust for padding
        x0 = clip((x0 - left / network_in_size) / (1 -  2 * left / network_in_size))
        x1 = clip((x1 - left / network_in_size) / (1 -  2 * left / network_in_size))
        y0 = clip((y0 - top / network_in_size) / (1 -  2 * top / network_in_size))
        y1 = clip((y1 - top / network_in_size) / (1 -  2 * top / network_in_size))
        # convert to pixels
        x0 = round(x0*image_w)
        x1 = round(x1*image_w)
        y0 = round(y0*image_h)
        y1 = round(y1*image_h)
        
        result.append({
            'box':[x0,y0,x1,y1]
            })
    return result

In [11]:
outputs = transform_predictions(boxes, scores, valid_detections,w,h, top, left)

In [12]:
im = Image.open(path)
draw = ImageDraw.Draw(im)

for detection in outputs[:10]:
    bb = detection['box']
    draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
               (bb[2], bb[1]), (bb[0], bb[1])], width=4, fill=(255, 0, 0))
im.show()